In [28]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, ToolMessage, AIMessage
from langchain_core.messages.human import HumanMessage
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
import os

In [29]:
tool = TavilySearchResults(
    max_results=2
) # 2 results per query
print (type(tool))
print (tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [30]:
# AgentState 类定义了代理的状态数据结构
# 这是一个 TypedDict 类型的类,用于定义具有类型提示的字典
# 包含一个 messages 字段:
# - messages 字段是一个列表,存储 AnyMessage 类型的消息
# - 使用 Annotated 和 operator.add 标注,表示该字段支持列表拼接操作
# - AnyMessage 是一个联合类型,可以是各种消息类型(AIMessage、HumanMessage等)
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

我们需要一个函数来调用`Ollama`，一个函数来检查是否存在某种action，以及一个执行action的函数。

In [31]:
# 配置日志记录系统
import logging

# 设置基本的日志配置
# level=logging.INFO 表示将捕获 INFO 级别及以上的所有日志消息
# 日志级别从低到高: DEBUG < INFO < WARNING < ERROR < CRITICAL
logging.basicConfig(level=logging.INFO)

# 创建一个日志记录器实例
# __name__ 是当前模块的名称
# 这个日志记录器将被 Agent 类用来记录执行过程中的信息
logger = logging.getLogger(__name__)
import sys
from datetime import datetime
from pathlib import Path

# 创建logs目录（如果不存在）
log_dir = Path("./logs")
log_dir.mkdir(exist_ok=True)

# 生成日志文件名（包含时间戳）
log_file = log_dir / f"agent_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

# 配置日志格式
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# 文件处理器
file_handler = logging.FileHandler(log_file)
file_handler.setFormatter(formatter)
file_handler.setLevel(logging.INFO)

# 控制台处理器（带颜色）
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(logging.Formatter(
    '%(asctime)s - \033[1;36m%(levelname)s\033[0m - %(message)s'))
console_handler.setLevel(logging.INFO)

# 配置logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(file_handler)
logger.addHandler(console_handler)

In [32]:
class Agent:
    """
    Agent class that orchestrates interaction between a language model and tools.
    Uses a state graph to manage conversation flow and decision making process.
    """

    def __init__(self, model, tools, system=""):
        """
        Initialize the agent with a language model, tools, and optional system message.
        
        Args:
            model: Language model that will generate responses
            tools: List of tools the agent can use
            system: Optional system prompt to guide agent behavior
            
        Creates a state graph with the following workflow:
            1. Start with LLM node
            2. Check if action is needed
            3. Execute action if needed, then return to LLM
            4. End conversation if no action is needed
        """
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
        logger.info(f"model: {self.model}")

    def exists_action(self, state: AgentState):
        """
        Determine if an action needs to be taken based on the last message.
        
        Args:
            state: Current conversation state containing messages
            
        Returns:
            Boolean indicating if the latest AI message contains tool calls
            that need to be executed
        """
        result = state['messages'][-1]
        logger.info(f"exists_action result: {result}")
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        """
        Send the current conversation state to the language model and get a response.
        
        Args:
            state: Current conversation state containing message history
            
        Returns:
            Updated state with the new AI message appended
            
        If a system prompt is provided, it's added at the beginning of messages
        before sending to the model.
        """
        logger.info(f"state: {state}")
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        logger.info(f"LLM message: {message}")
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        """
        Execute tool calls requested by the language model.
        
        Args:
            state: Current conversation state containing message history
            
        Returns:
            Updated state with tool execution results appended as ToolMessages
            
        Process:
        1. Extract tool calls from the last message
        2. For each tool call, validate the tool name and execute it
        3. Format results as ToolMessages with appropriate IDs
        4. Return all results to be processed by the model in the next step
        """
        import threading
        print(f"take_action called in thread: {threading.current_thread().name}")
        tool_calls = state['messages'][-1].tool_calls
        results = []
        print(f"take_action called with tool_calls: {tool_calls}")
        for t in tool_calls:
            logger.info(f"Calling: {t}")
            print(f"Calling: {t}") 
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                logger.info(f"action {t['name']}, result: {result}")
                print(f"action {t['name']}, result: {result}")
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [33]:
prompt = """You are a smart research assistant. Use the search engine to look up information.  \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOllama(
    model="qwen2.5:7b",
    temperature=0
)
abot = Agent(model, [tool], system=prompt)

2025-03-08 18:21:56,604 - INFO - model: bound=ChatOllama(model='qwen2.5:7b', temperature=0.0) kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]} config={} config_factories=[]


INFO:__main__:model: bound=ChatOllama(model='qwen2.5:7b', temperature=0.0) kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]} config={} config_factories=[]


In [34]:
messages = [
    HumanMessage(
        content="What is the weather in sf?"
    )
]
result = abot.graph.invoke({'messages': messages})

2025-03-08 18:21:56,611 - INFO - state: {'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={})]}


INFO:__main__:state: {'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={})]}
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-03-08 18:22:01,709 - INFO - LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:01.706173Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5087081333, 'load_duration': 563421083, 'prompt_eval_count': 244, 'prompt_eval_duration': 3201000000, 'eval_count': 26, 'eval_duration': 1087000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-4af02844-7329-4166-a069-ffc673b749c1-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '4943bf01-21d4-4d92-bc07-0d24c4433b34', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}


INFO:__main__:LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:01.706173Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5087081333, 'load_duration': 563421083, 'prompt_eval_count': 244, 'prompt_eval_duration': 3201000000, 'eval_count': 26, 'eval_duration': 1087000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-4af02844-7329-4166-a069-ffc673b749c1-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '4943bf01-21d4-4d92-bc07-0d24c4433b34', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}


2025-03-08 18:22:01,713 - INFO - exists_action result: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:01.706173Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5087081333, 'load_duration': 563421083, 'prompt_eval_count': 244, 'prompt_eval_duration': 3201000000, 'eval_count': 26, 'eval_duration': 1087000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-4af02844-7329-4166-a069-ffc673b749c1-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '4943bf01-21d4-4d92-bc07-0d24c4433b34', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}


...

INFO:__main__:state: {'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:01.706173Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5087081333, 'load_duration': 563421083, 'prompt_eval_count': 244, 'prompt_eval_duration': 3201000000, 'eval_count': 26, 'eval_duration': 1087000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-4af02844-7329-4166-a069-ffc673b749c1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '4943bf01-21d4-4d92-bc07-0d24c4433b34', 'type': 'tool_call'}], usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}), ToolMessage(content='[{\'title\': \'Weather in san francisco\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'

2025-03-08 18:22:10,890 - INFO - LLM message: content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently observed.\n\nFor more detailed information or future forecasts, you can visit [WeatherAPI](https://www.weatherapi.com/)." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:10.878585Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7379557458, 'load_duration': 39138458, 'prompt_eval_count': 1022, 'prompt_eval_duration': 3745000000, 'eval_count': 81, 'eval_duration': 3573000000, 'message': Message(role='assistant', content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently observed.\n\n

INFO:__main__:LLM message: content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently observed.\n\nFor more detailed information or future forecasts, you can visit [WeatherAPI](https://www.weatherapi.com/)." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:10.878585Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7379557458, 'load_duration': 39138458, 'prompt_eval_count': 1022, 'prompt_eval_duration': 3745000000, 'eval_count': 81, 'eval_duration': 3573000000, 'message': Message(role='assistant', content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently observed.\n\nFor more detailed i

2025-03-08 18:22:10,892 - INFO - exists_action result: content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently observed.\n\nFor more detailed information or future forecasts, you can visit [WeatherAPI](https://www.weatherapi.com/)." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:10.878585Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7379557458, 'load_duration': 39138458, 'prompt_eval_count': 1022, 'prompt_eval_duration': 3745000000, 'eval_count': 81, 'eval_duration': 3573000000, 'message': Message(role='assistant', content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently obse

INFO:__main__:exists_action result: content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently observed.\n\nFor more detailed information or future forecasts, you can visit [WeatherAPI](https://www.weatherapi.com/)." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-08T10:22:10.878585Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7379557458, 'load_duration': 39138458, 'prompt_eval_count': 1022, 'prompt_eval_duration': 3745000000, 'eval_count': 81, 'eval_duration': 3573000000, 'message': Message(role='assistant', content="The current weather in San Francisco, CA is partly cloudy with a temperature of 7.2°C (45°F). The wind speed is at 2.2 mph coming from the southwest direction. The humidity level is 90%, and there's no precipitation currently observed.\n\nFor more d